# Posterior and Marginal distributions

This notebook is the continuation of `bandwidths.ipynb` notebook, apliying the bw results and the class for get the n-dimentional PDF. 

#### Some considerations: 
1. As **prior** information we will take the cleaned (without NaN or inf values) information from exoplanet.eu, this is the first part of notebook.

2. The PDF from `oiptimal_pdf` class  fulfills the functions of **likelihood** for a certain number of variables in synthetic systems with no-pertutbation, low perturbation and high perturbation. 

3. To get the **marginal** distributions of a variable of interest, we go in the same way that the example marginalization in the notebook `3D.ipynb`.

In [10]:
import numpy as np
import pandas as pd
import warnings; warnings.simplefilter('ignore')

import nbimporter
from bandwidths import optimal_pdf #import the class for get the pdf.

In [18]:
from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

## Data Cleaning

### 1. Simulation data   

see the notation according to <a href="https://github.com/saint-germain/population_synthesis/blob/master/README.md ">Readme.md</a> from `Population_synthesis` project.





In [15]:
#========================== Simulation Data ==========================
dn=pd.read_csv('data/proc_no_pert.csv',index_col=None); dn["gia"]=dn.ngi>0 #without pertubtations 
dl=pd.read_csv('data/proc_lo_pert.csv',index_col=None); dl["gia"]=dl.ngi>0 #with low pertubtations 
dh=pd.read_csv('data/proc_hi_pert.csv',index_col=None); dh["gia"]=dh.ngi>0 #with high pertubtations

In [16]:
#======================= Simulation variables ========================
##Terrestrial: t; giant;g
dnt=dn[~dn["gia"]]; dng=dn[dn["gia"]] # without pertubtations 
dlt=dl[~dl["gia"]]; dlg=dl[dl["gia"]] # low pertubtations 
dht=dh[~dh["gia"]]; dhg=dh[dh["gia"]] # high pertubtations 

x_variables = [dng,dlg,dhg,dnt,dlt,dht,dn,dl,dh]

for i, var in enumerate(x_variables):
    var['logeff'] = np.log10(var.massefficiency)
    var['logcom'] = np.log10(var.com)

In [17]:
dnt.head()

,Unnamed: 0,ident,com,nplanets,massbudget,massefficiency,sigmag0,md,rc,ms,metal,taugas,qest,ngi,mtr,apert,gia,logeff,logcom
1,1,5.0,2.932894,12.0,17.882769,0.000488,102.431593,0.110,38.977428,1.075269,-0.150160,1.014449e+06,5.464831,0.0,17.882769,0.0,False,-3.311837,0.467296
3,3,8.0,5.740174,9.0,8.166382,0.000163,62.737337,0.150,58.158928,1.076658,-0.282408,6.017040e+06,4.704798,0.0,8.166382,0.0,False,-3.786940,0.758925
5,5,15.0,8.394027,8.0,16.003091,0.000436,106.824759,0.110,38.167542,0.986003,0.388613,2.435406e+06,5.218175,0.0,16.003091,0.0,False,-3.360068,0.923970
6,6,16.0,4.289089,24.0,12.426573,0.000219,118.543720,0.170,45.042137,1.258747,-0.352459,1.107032e+06,4.469478,0.0,12.426573,0.0,False,-3.658976,0.632365
7,7,17.0,3.771156,12.0,16.762554,0.000811,35.587738,0.062,49.645451,0.739731,0.121866,9.050091e+06,7.257983,0.0,16.762554,0.0,False,-3.090930,0.576474


### 2. Observational data 

Data get from <a href="http://exoplanet.eu/">exoplanet.eu</a>

In [48]:
data_o = pd.read_csv('data/exoplanet.eu_catalog.csv', 
                       usecols = ['mass','mass_error_min', 
                                  'semi_major_axis', 'semi_major_axis_error_min', 
                                  'star_metallicity', 'star_metallicity_error_min', 
                                  'star_mass', 'star_mass_error_min', 'star_name'])

data_o = data_o.replace([np.inf, -np.inf], np.nan) 
data_o = data_o.replace([0], np.nan)
data_o = data_o.dropna()

In [49]:
data_o = data_o[['star_name','mass','mass_error_min', 'semi_major_axis', 'semi_major_axis_error_min', 
                 'star_metallicity', 'star_metallicity_error_min', 'star_mass', 'star_mass_error_min']]

In [50]:
p_system = data_o.groupby("star_name")
number=(data_o["star_name"].value_counts()).to_frame()

In [52]:
def CoM(data):
    data = data.assign(CM_i=data["semi_major_axis"]*data["mass"])
    p_system = data.groupby("star_name")
    
    CoM = self.p_system['CM_i'].sum().divide(self.p_system["mass"].sum())
    self.NewData = pd.DataFrame({'System_name':list(self.p_system.groups.keys()),
                                 'Total_mass':self.p_system["mass"].sum().tolist(),
                                 'Center_of_Mass':CoM.tolist()})

In [33]:
data_o

,star_name,mass,mass_error_min,semi_major_axis,semi_major_axis_error_min,star_metallicity,star_metallicity_error_min,star_mass,star_mass_error_min
58,55 Cnc,0.84000,0.03100,0.113390,0.000110,0.31,0.04,1.015,0.051
61,55 Cnc,0.02703,0.00135,0.015439,0.000015,0.31,0.04,1.015,0.051
86,BD+20 594,0.05130,0.01900,0.241000,0.019000,-0.15,0.05,0.961,0.029
102,CD Cet,0.01244,0.00136,0.018500,0.001300,0.13,0.16,0.161,0.010
103,CD-35 2722,31.00000,8.00000,67.000000,4.000000,0.04,0.50,0.400,0.050
...,...,...,...,...,...,...,...,...,...
4337,gamma 1 Leo,63.88000,56.10000,1.190000,0.020000,-0.51,0.05,1.230,0.210
4341,kappa And,13.00000,2.00000,100.000000,46.000000,-0.36,0.09,2.800,0.200
4355,pi Men,13.01000,1.00000,3.308000,0.039000,0.08,0.03,1.094,0.039
4356,pi Men,0.01517,0.00267,0.067020,0.000500,0.08,0.03,1.094,0.039


In [30]:
data_o.head()

,mass,mass_error_min,semi_major_axis,semi_major_axis_error_min,star_name,star_metallicity,star_metallicity_error_min,star_mass,star_mass_error_min
58,0.84000,0.03100,0.113390,0.000110,55 Cnc,0.31,0.04,1.015,0.051
61,0.02703,0.00135,0.015439,0.000015,55 Cnc,0.31,0.04,1.015,0.051
86,0.05130,0.01900,0.241000,0.019000,BD+20 594,-0.15,0.05,0.961,0.029
102,0.01244,0.00136,0.018500,0.001300,CD Cet,0.13,0.16,0.161,0.010
103,31.00000,8.00000,67.000000,4.000000,CD-35 2722,0.04,0.50,0.400,0.050


In [8]:
a = multidim_bw(dng.logeff, dng.logcom)
likelihood = a.pdf_ndim()

NameError: name 'multidim_bw' is not defined